### Importing Libraries

In [5]:
import numpy as np
import pandas as pd

import psycopg2 as pg2
from psycopg2.extras import RealDictCursor, Json

In [6]:
#function to connect to posgres db on ubuntu instance 

def con_cur_to_db():
    con = pg2.connect(host = '13.52.0.109', 
                     dbname = 'postgres',
                     user = 'postgres',
                     password = 'zoolander')
    cur = con.cursor(cursor_factory = RealDictCursor)
    return con, cur

In [21]:
#function to insert data into unstructured postgres db

def insert_json(data):
    con, cur = con_cur_to_db()
    for x in data:
        cur.execute('INSERT INTO all_data (data) VALUES ({});'.format(Json(x)))
    con.commit()
    con.close()

In [22]:
insert_json(tweet_data)

In [ ]:
prime_labels = ['gucci', 'McQueen', 'StellaMcCartney', 'Roberto_Cavalli', 'acnestudios', 'jimmychoo', 'Roberto_Cavalli',\
               'TOMFORD', 'Prada', 'akris_official']

In [ ]:

for label in prime_labels:
    tweet_data = miner.mine_user_tweets(user=label, max_pages=500, last_tweet_id=True)
    
insert_json(tweet_data)

### Collecting Fashion House Tweets

In [7]:
#Reading in application keys 

with open('../data/twitter_keys.txt') as api:
    keys = api.read().split('\n') 

#Aplication keys

import twitter, re, datetime, pandas as pd

twitter_keys = {
    'consumer_key':        keys[0],
    'consumer_secret':     keys[1],
    'access_token_key':    keys[2],
    'access_token_secret': keys[3]
}

api = twitter.Api(
    consumer_key         =   twitter_keys['consumer_key'],
    consumer_secret      =   twitter_keys['consumer_secret'],
    access_token_key     =   twitter_keys['access_token_key'],
    access_token_secret  =   twitter_keys['access_token_secret'],
    tweet_mode           =   'extended'
)


In [9]:
statuses = api.GetUserTimeline(screen_name="gucci", count=200, trim_user=False)

In [ ]:
tweet = statuses[0]
tweet.full_text

In [11]:
[status.full_text for status in statuses]

['A fresh and green scent Gucci Bloom Acqua di Fiori, a new chapter of the Gucci Bloom #InBloom fragrance story, captured in a campaign featuring #DakotaJohnson and @petracollins https://t.co/El1buBhuCU \nPhoto by: #GlenLunchford\nArt director: #ChristopherSimmonds\n#AlessandroMichele https://t.co/dVNtSosZ0J',
 'New scent Gucci Bloom Acqua di Fiori blends a watery accord of green Galbanum and dainty Cassis buds with the flourishing garden perfume from the original fragrance https://t.co/El1buBhuCU #InBloom \nPhoto by: #GlenLunchford\nArt director: #ChristopherSimmonds\n#AlessandroMichele https://t.co/lUgkB1Hvc0',
 'Presenting a new chapter of #InBloom, Gucci Bloom Acqua di Fiori, a fresh green scent in a campaign featuring #DakotaJohnson, @harinef and @petracollins https://t.co/El1buBhuCU \nPhotographer: #GlenLunchford\nArt director: #ChristopherSimmonds\nCreative director: #AlessandroMichele https://t.co/xkTe1Mtvlk',
 '“It was my way of surviving, to charm people or even confuse them 

In [12]:
#Twitter API Class

class TweetMiner():
    
    result_limit = 20
    api          = None
    tweets       = []
    
    def __init__(self, keys_dict, api, result_limit = 20):
        
        self.keys_dict     = keys_dict
        self.api           = api
        self.results_limit = result_limit
        
    def mine_user_tweets(self, user='gucci', mine_retweets = False, max_pages = None, last_tweet_id = False):
        
        #time out feedback
        print("Mining tweets for:", user)
        
        tweets        = []
        
        page           = 1
        
        while page <= max_pages:
            
            #time out feedback
            print("Mining page:", page)
            
            if last_tweet_id:
                statuses = self.api.GetUserTimeline(
                    screen_name = user,
                    count       = self.result_limit,
                    max_id      = last_tweet_id - 1
                )
            else:
                statuses = self.api.GetUserTimeline(
                    screen_name = user,
                    count       = self.result_limit
                )

            for item in statuses:
                
                last_tweet_id = item.id
                
                tweets.append({
                    'tweet_id': item.id,
                    'handle':   item.user.name,
                    'text':     item.full_text,
                    'created':  item.created_at
                })
                
            page += 1
        
        return tweets
            
        
miner = TweetMiner(twitter_keys, api, result_limit=50)

### Fashion Labels

In [13]:
wikipedia_labels = ['akris_official', 'McQueen', 'alfreddunhill', 'AlphaIndustries', 'andres_sarda', 'Anne_Fontaine',\
                   'aquascutum', 'giorgioarmani', 'emporioarmani', 'ArmaniExchange', 'AspinalofLondon', 'AspreyLondon',\
                   'AleDellAcqua', 'Dior', 'dkny', 'dolcegabbana', '_BadgleyMischka', 'BadgleyMischka', 'BALENCIAGA',\
                   'Bally', 'Balmain', 'BensonAndClegg', 'berluti', 'Beulahlondon', 'ManoloBlahnik', 'HUGOBOSS',\
                    'BottegaVeneta', 'ThomBrowneNY', 'Bulgariofficial', 'Burberry', '']

In [27]:
fashion_labels = ['gucci', 'McQueen', 'StellaMcCartney', 'Fendi', 'MaisonValentino', 'Roberto_Cavalli',\
                  'LouboutinWorld', 'EtroOfficial', 'Prada', 'dolcegabbana', 'Dior', 'LouisVuitton', 'CHANEL'\
                 'Missoni', 'OscardelaRenta', 'VeraWangGang', 'HerveLeger', 'MaisonValentino', 'Balmain',\
                 'Versace', 'giorgioarmani', 'DerekLamNYC', 'AlbertaFerretti', 'YSL', 'COMMEGARCONS', 'BALENCIAGA',\
                 'Burberry', 'TOMFORD', 'ALEXVAUTHIER', 'MIUMIUofficial', 'CalvinKlein', 'givenchy', 'TommyHilfiger',\
                 'jimmychoo', 'OffWht', 'chloefashion','dkny', 'marcjacobs', 'MichaelKors', 'Goyard', 'HUGOBOSS',\
                 'kenzo', 'HELMUTLANG', 'acnestudios', 'Bulgariofficial', 'akris_official', '']

prime_labels = ['gucci', 'McQueen', 'StellaMcCartney', 'Roberto_Cavalli', 'acnestudios', 'jimmychoo', 'Roberto_Cavalli',\
               'TOMFORD', 'Prada', 'akris_official']

print('fashion_labels:', len(fashion_labels))
print('prime_labels:', len(prime_labels))
print('wikipedia_labels', len(wikipedia_labels))

fashion_labels: 46
prime_labels: 8
wikipedia_labels 31


In [16]:
tweet_data = miner.mine_user_tweets(user='gucci', max_pages=50, last_tweet_id=True)

Mining tweets for: gucci
Mining page: 1
Mining page: 2
Mining page: 3
Mining page: 4
Mining page: 5
Mining page: 6
Mining page: 7
Mining page: 8
Mining page: 9
Mining page: 10
Mining page: 11
Mining page: 12
Mining page: 13
Mining page: 14
Mining page: 15
Mining page: 16
Mining page: 17
Mining page: 18
Mining page: 19
Mining page: 20
Mining page: 21
Mining page: 22
Mining page: 23
Mining page: 24
Mining page: 25
Mining page: 26
Mining page: 27
Mining page: 28
Mining page: 29
Mining page: 30
Mining page: 31
Mining page: 32
Mining page: 33
Mining page: 34
Mining page: 35
Mining page: 36
Mining page: 37
Mining page: 38
Mining page: 39
Mining page: 40
Mining page: 41
Mining page: 42
Mining page: 43
Mining page: 44
Mining page: 45
Mining page: 46
Mining page: 47
Mining page: 48
Mining page: 49
Mining page: 50


In [20]:
tweet_data[0]

{'created': 'Wed Mar 21 22:00:19 +0000 2018',
 'handle': 'gucci',
 'text': 'A fresh and green scent Gucci Bloom Acqua di Fiori, a new chapter of the Gucci Bloom #InBloom fragrance story, captured in a campaign featuring #DakotaJohnson and @petracollins https://t.co/El1buBhuCU \nPhoto by: #GlenLunchford\nArt director: #ChristopherSimmonds\n#AlessandroMichele https://t.co/dVNtSosZ0J',
 'tweet_id': 976579281898885120}

In [28]:
#Instantiating class and passing list of twitter handles

tweet_data = []

for label in prime_labels:
    try:
        tweet_data = miner.mine_user_tweets(user=label, max_pages=500, last_tweet_id=True)
        df = pd.DataFrame(tweet_data)
        df.to_csv('../data/%s.csv' % label)
    except:
        print('could not mine label:', label)


Mining tweets for: StellaMcCartney
Mining page: 1
Mining page: 2
Mining page: 3
Mining page: 4
Mining page: 5
Mining page: 6
Mining page: 7
Mining page: 8
Mining page: 9
Mining page: 10
Mining page: 11
Mining page: 12
Mining page: 13
Mining page: 14
Mining page: 15
Mining page: 16
Mining page: 17
Mining page: 18
Mining page: 19
Mining page: 20
Mining page: 21
Mining page: 22
Mining page: 23
Mining page: 24
Mining page: 25
Mining page: 26
Mining page: 27
Mining page: 28
Mining page: 29
Mining page: 30
Mining page: 31
Mining page: 32
Mining page: 33
Mining page: 34
Mining page: 35
Mining page: 36
Mining page: 37
Mining page: 38
Mining page: 39
Mining page: 40
Mining page: 41
Mining page: 42
Mining page: 43
Mining page: 44
Mining page: 45
Mining page: 46
Mining page: 47
Mining page: 48
Mining page: 49
Mining page: 50
Mining page: 51
Mining page: 52
Mining page: 53
Mining page: 54
Mining page: 55
Mining page: 56
Mining page: 57
Mining page: 58
Mining page: 59
Mining page: 60
Mining page: 6

Mining page: 488
Mining page: 489
Mining page: 490
Mining page: 491
Mining page: 492
Mining page: 493
Mining page: 494
Mining page: 495
Mining page: 496
Mining page: 497
Mining page: 498
Mining page: 499
Mining page: 500
Mining tweets for: Roberto_Cavalli
Mining page: 1
Mining page: 2
Mining page: 3
Mining page: 4
Mining page: 5
Mining page: 6
Mining page: 7
Mining page: 8
Mining page: 9
Mining page: 10
Mining page: 11
Mining page: 12
Mining page: 13
Mining page: 14
Mining page: 15
Mining page: 16
Mining page: 17
Mining page: 18
Mining page: 19
Mining page: 20
Mining page: 21
Mining page: 22
Mining page: 23
Mining page: 24
Mining page: 25
Mining page: 26
Mining page: 27
Mining page: 28
Mining page: 29
Mining page: 30
Mining page: 31
Mining page: 32
Mining page: 33
Mining page: 34
Mining page: 35
Mining page: 36
Mining page: 37
Mining page: 38
Mining page: 39
Mining page: 40
Mining page: 41
Mining page: 42
Mining page: 43
Mining page: 44
Mining page: 45
Mining page: 46
Mining page: 47
M

In [12]:
#Instantiating class and passing list of twitter handles

tweet_data = []

for label in prime_labels:
    tweet_data.extend(miner.mine_user_tweets(user=label, max_pages=50, last_tweet_id=True))

fashion = pd.DataFrame(tweet_data)

Mining tweets for: gucci
Mining page: 1
Mining page: 2
Mining page: 3
Mining page: 4
Mining page: 5
Mining page: 6
Mining page: 7
Mining page: 8
Mining page: 9
Mining page: 10
Mining page: 11
Mining page: 12
Mining page: 13
Mining page: 14
Mining page: 15
Mining page: 16
Mining page: 17
Mining page: 18
Mining page: 19
Mining page: 20
Mining page: 21
Mining page: 22
Mining page: 23
Mining page: 24
Mining page: 25
Mining page: 26
Mining page: 27
Mining page: 28
Mining page: 29
Mining page: 30
Mining page: 31
Mining page: 32
Mining page: 33
Mining page: 34
Mining page: 35
Mining page: 36
Mining page: 37
Mining page: 38
Mining page: 39
Mining page: 40
Mining page: 41
Mining page: 42
Mining page: 43
Mining page: 44
Mining page: 45
Mining page: 46
Mining page: 47
Mining page: 48
Mining page: 49
Mining page: 50
Mining tweets for: McQueen
Mining page: 1
Mining page: 2
Mining page: 3
Mining page: 4
Mining page: 5
Mining page: 6
Mining page: 7
Mining page: 8
Mining page: 9
Mining page: 10
Mining

In [14]:
fashion.shape

(9998, 4)

In [26]:
fashion.groupby(fashion.handle).size()

NameError: name 'fashion' is not defined

In [16]:
fashion.to_pickle('../data/fashion.pkl')

In [17]:
fashion.head()

,created,handle,text,tweet_id
0,Wed Mar 21 22:00:19 +0000 2018,gucci,A fresh and green scent Gucci Bloom Acqua di F...,976579281898885120
1,Wed Mar 21 20:00:19 +0000 2018,gucci,New scent Gucci Bloom Acqua di Fiori blends a ...,976549082427740160
2,Wed Mar 21 17:59:41 +0000 2018,gucci,"Presenting a new chapter of #InBloom, Gucci Bl...",976518722830716931
3,Tue Mar 20 19:00:35 +0000 2018,gucci,"“It was my way of surviving, to charm people o...",976171664982659073
4,Tue Mar 20 16:00:39 +0000 2018,gucci,".@MichaelClarkCo reveals, ""Japan is the most a...",976126380206383104


In [25]:
pd.read_csv('../data/gucci.csv', index_col=[0]).info()

<class 'pandas.core.frame.DataFrame'>
Index: 3241 entries, 0 to 3233
Data columns (total 4 columns):
created     3241 non-null object
handle      3234 non-null object
text        3234 non-null object
tweet_id    3227 non-null float64
dtypes: float64(1), object(3)
memory usage: 126.6+ KB


In [29]:
pd.read_csv('../data/StellaMcCartney.csv', index_col=[0]).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3219 entries, 0 to 3218
Data columns (total 4 columns):
created     3219 non-null object
handle      3219 non-null object
text        3219 non-null object
tweet_id    3219 non-null int64
dtypes: int64(1), object(3)
memory usage: 125.7+ KB


In [30]:
pd.read_csv('../data/Roberto_Cavalli.csv', index_col=[0]).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 4 columns):
created     1000 non-null object
handle      1000 non-null object
text        1000 non-null object
tweet_id    1000 non-null int64
dtypes: int64(1), object(3)
memory usage: 39.1+ KB


In [34]:
pd.read_csv('../data/McQueen.csv', index_col=[0]).info()

<class 'pandas.core.frame.DataFrame'>
Index: 1002 entries, 0 to 999
Data columns (total 4 columns):
created     1002 non-null object
handle      1000 non-null object
text        1000 non-null object
tweet_id    998 non-null float64
dtypes: float64(1), object(3)
memory usage: 39.1+ KB
